# Task 7: AutoFeatureSelector Tool
## This task is to test your understanding of various Feature Selection methods outlined in the lecture and the ability to apply this knowledge in a real-world dataset to select best features and also to build an automated feature selection tool as your toolkit

### Use your knowledge of different feature selector methods to build an Automatic Feature Selection tool
- Pearson Correlation
- Chi-Square
- RFE
- Embedded
- Tree (Random Forest)
- Tree (Light GBM)

### Dataset: FIFA 19 Player Skills
#### Attributes: FIFA 2019 players attributes like Age, Nationality, Overall, Potential, Club, Value, Wage, Preferred Foot, International Reputation, Weak Foot, Skill Moves, Work Rate, Position, Jersey Number, Joined, Loaned From, Contract Valid Until, Height, Weight, LS, ST, RS, LW, LF, CF, RF, RW, LAM, CAM, RAM, LM, LCM, CM, RCM, RM, LWB, LDM, CDM, RDM, RWB, LB, LCB, CB, RCB, RB, Crossing, Finishing, Heading, Accuracy, ShortPassing, Volleys, Dribbling, Curve, FKAccuracy, LongPassing, BallControl, Acceleration, SprintSpeed, Agility, Reactions, Balance, ShotPower, Jumping, Stamina, Strength, LongShots, Aggression, Interceptions, Positioning, Vision, Penalties, Composure, Marking, StandingTackle, SlidingTackle, GKDiving, GKHandling, GKKicking, GKPositioning, GKReflexes, and Release Clause.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as ss
from collections import Counter
import math
from scipy import stats

In [2]:
player_df = pd.read_csv("fifa19.csv")

In [4]:
player_df.columns

Index(['Unnamed: 0', 'ID', 'Name', 'Age', 'Photo', 'Nationality', 'Flag',
       'Overall', 'Potential', 'Club', 'Club Logo', 'Value', 'Wage', 'Special',
       'Preferred Foot', 'International Reputation', 'Weak Foot',
       'Skill Moves', 'Work Rate', 'Body Type', 'Real Face', 'Position',
       'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until',
       'Height', 'Weight', 'LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW',
       'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM',
       'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'Crossing',
       'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling',
       'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingT

In [3]:
numcols = ['Overall', 'Crossing','Finishing',  'ShortPassing',  'Dribbling','LongPassing', 'BallControl', 'Acceleration','SprintSpeed', 'Agility',  'Stamina','Volleys','FKAccuracy','Reactions','Balance','ShotPower','Strength','LongShots','Aggression','Interceptions']
catcols = ['Preferred Foot','Position','Body Type','Nationality','Weak Foot']

In [5]:
player_df = player_df[numcols+catcols]

In [7]:
pd.get_dummies(player_df[catcols]).columns

Index(['Weak Foot', 'Preferred Foot_Left', 'Preferred Foot_Right',
       'Position_CAM', 'Position_CB', 'Position_CDM', 'Position_CF',
       'Position_CM', 'Position_GK', 'Position_LAM',
       ...
       'Nationality_Uganda', 'Nationality_Ukraine',
       'Nationality_United Arab Emirates', 'Nationality_United States',
       'Nationality_Uruguay', 'Nationality_Uzbekistan',
       'Nationality_Venezuela', 'Nationality_Wales', 'Nationality_Zambia',
       'Nationality_Zimbabwe'],
      dtype='object', length=204)

In [30]:
traindf = pd.concat([player_df[numcols], pd.get_dummies(player_df[catcols])],axis=1)
features = traindf.columns

In [31]:
traindf.shape

(18207, 224)

In [33]:
traindf = traindf.dropna()

In [34]:
traindf.shape

(18159, 224)

In [47]:
y = traindf['Overall']>=87
X = traindf.drop('Overall',axis=1)

In [36]:
X.shape, y.shape

((18159, 223), (18159,))

### Set some fixed set of features

In [37]:
feature_name = list(X.columns)

# no of maximum features we need to select
num_feats=30

## Filter Feature Selection - Pearson Correlation

### Pearson Correlation function

In [38]:
# for example: 
np.corrcoef(X['Weak Foot'], y)

array([[1.        , 0.21201545],
       [0.21201545, 1.        ]])

In [51]:
def cor_selector(X, y,num_feats):
    
    cor_list = []
    for i in feature_name:
        cor = np.corrcoef(X[i], y)[0, 1]
        cor_list.append(cor)
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    cor_list=np.abs(cor_list)
    #correlation coefficient list 
    
    order_in_index=np.argsort(cor_list)    #least to largest, ordering in their indexes

    numfeats=order_in_index[-num_feats:]      #30 indexes with largest values

    cor_feature = X.iloc[:,numfeats].columns.tolist()
    
    cor_support = [True if i in cor_feature else False for i in feature_name]      
    
    return cor_support, cor_feature

In [52]:
cor_support, cor_feature = cor_selector(X, y,num_feats)
print(str(len(cor_feature)), 'selected features')

30 selected features


### List the selected features from Pearson Correlation

In [53]:
cor_feature

['Nationality_Costa Rica',
 'Position_LAM',
 'Nationality_Uruguay',
 'Acceleration',
 'SprintSpeed',
 'Strength',
 'Nationality_Gabon',
 'Nationality_Slovenia',
 'Stamina',
 'Weak Foot',
 'Agility',
 'Crossing',
 'Nationality_Belgium',
 'Dribbling',
 'ShotPower',
 'LongShots',
 'Finishing',
 'BallControl',
 'FKAccuracy',
 'LongPassing',
 'Volleys',
 'ShortPassing',
 'Position_RF',
 'Position_LF',
 'Body Type_Neymar',
 'Body Type_Messi',
 'Body Type_Courtois',
 'Body Type_PLAYER_BODY_TYPE_25',
 'Body Type_C. Ronaldo',
 'Reactions']

## Filter Feature Selection - Chi-Sqaure

In [64]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

### Chi-Squared Selector function

In [65]:
def chi_squared_selector(X, y, num_feats):
    
    X_norm=MinMaxScaler().fit_transform(X)
    chi_selector=SelectKBest(score_func=chi2, k=num_feats)
    chi_selector.fit(X_norm,y)
    chi_support=chi_selector.get_support()
    chi_feature=X.loc[:,chi_support].columns.tolist()     
    
    return chi_support, chi_feature

In [66]:
chi_support, chi_feature = chi_squared_selector(X, y,num_feats)
print(str(len(chi_feature)), 'selected features')

30 selected features


### List the selected features from Chi-Square 

In [67]:
chi_feature

['Finishing',
 'ShortPassing',
 'LongPassing',
 'BallControl',
 'Volleys',
 'FKAccuracy',
 'Reactions',
 'LongShots',
 'Position_CM',
 'Position_LAM',
 'Position_LF',
 'Position_LW',
 'Position_RB',
 'Position_RF',
 'Body Type_C. Ronaldo',
 'Body Type_Courtois',
 'Body Type_Messi',
 'Body Type_Neymar',
 'Body Type_PLAYER_BODY_TYPE_25',
 'Nationality_Belgium',
 'Nationality_Costa Rica',
 'Nationality_Croatia',
 'Nationality_Egypt',
 'Nationality_England',
 'Nationality_France',
 'Nationality_Gabon',
 'Nationality_Slovakia',
 'Nationality_Slovenia',
 'Nationality_Spain',
 'Nationality_Uruguay']

## Wrapper Feature Selection - Recursive Feature Elimination

In [68]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

### RFE Selector function

In [69]:
def rfe_selector(X, y, num_feats):

    lr = LogisticRegression()
    rfe_selector=RFE(estimator=lr,n_features_to_select=num_feats,step=10,verbose=5)
    
    X_norm=MinMaxScaler().fit_transform(X)
    
    rfe_selector.fit(X_norm,y)
    rfe_support=rfe_selector.get_support()
    rfe_feature=X.loc[:,rfe_support].columns.tolist()      

    return rfe_support, rfe_feature

In [70]:
rfe_support, rfe_feature = rfe_selector(X, y,num_feats)
print(str(len(rfe_feature)), 'selected features')

Fitting estimator with 223 features.
Fitting estimator with 213 features.
Fitting estimator with 203 features.
Fitting estimator with 193 features.
Fitting estimator with 183 features.
Fitting estimator with 173 features.
Fitting estimator with 163 features.
Fitting estimator with 153 features.
Fitting estimator with 143 features.
Fitting estimator with 133 features.
Fitting estimator with 123 features.
Fitting estimator with 113 features.
Fitting estimator with 103 features.
Fitting estimator with 93 features.
Fitting estimator with 83 features.
Fitting estimator with 73 features.
Fitting estimator with 63 features.
Fitting estimator with 53 features.
Fitting estimator with 43 features.
Fitting estimator with 33 features.
30 selected features


### List the selected features from RFE

In [71]:
rfe_feature

['Finishing',
 'ShortPassing',
 'LongPassing',
 'BallControl',
 'SprintSpeed',
 'Agility',
 'Volleys',
 'FKAccuracy',
 'Reactions',
 'Strength',
 'Weak Foot',
 'Position_CAM',
 'Position_CM',
 'Position_GK',
 'Position_LCB',
 'Position_LM',
 'Position_RB',
 'Position_RCB',
 'Position_RF',
 'Position_RM',
 'Position_RW',
 'Body Type_Courtois',
 'Body Type_PLAYER_BODY_TYPE_25',
 'Nationality_Belgium',
 'Nationality_Costa Rica',
 'Nationality_Croatia',
 'Nationality_Gabon',
 'Nationality_Netherlands',
 'Nationality_Slovenia',
 'Nationality_Uruguay']

## Embedded Selection - Lasso: SelectFromModel

In [72]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

In [85]:
def embedded_log_reg_selector(X, y, num_feats):

    embedded_lr_selector=SelectFromModel(LogisticRegression(penalty='l2'),max_features=num_feats)
    
    X_norm=MinMaxScaler().fit_transform(X)
    
    embedded_lr_selector.fit(X_norm,y)
    embedded_lr_support=embedded_lr_selector.get_support()
    embedded_lr_feature=X.loc[:,embedded_lr_support].columns.tolist()       
    
    return embedded_lr_support, embedded_lr_feature

In [86]:
embedded_lr_support, embedded_lr_feature = embedded_log_reg_selector(X, y, num_feats)
print(str(len(embedded_lr_feature)), 'selected features')

30 selected features


In [87]:
embedded_lr_feature

['Finishing',
 'ShortPassing',
 'LongPassing',
 'BallControl',
 'SprintSpeed',
 'Agility',
 'Volleys',
 'Reactions',
 'Strength',
 'Weak Foot',
 'Position_CAM',
 'Position_CM',
 'Position_GK',
 'Position_LCB',
 'Position_LM',
 'Position_RB',
 'Position_RCB',
 'Position_RM',
 'Position_RW',
 'Body Type_Courtois',
 'Body Type_Lean',
 'Body Type_Stocky',
 'Nationality_Belgium',
 'Nationality_Costa Rica',
 'Nationality_Croatia',
 'Nationality_France',
 'Nationality_Gabon',
 'Nationality_Netherlands',
 'Nationality_Slovenia',
 'Nationality_Uruguay']

## Tree based(Random Forest): SelectFromModel

In [88]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

In [97]:
def embedded_rf_selector(X, y, num_feats):

    embedded_rf_selector=SelectFromModel(RandomForestClassifier(n_estimators=100),max_features=num_feats)

    X_norm=MinMaxScaler().fit_transform(X)

    embedded_rf_selector.fit(X_norm,y)
    embedded_rf_support=embedded_rf_selector.get_support()
    embedded_rf_feature=X.loc[:,embedded_rf_support].columns.tolist()    
    
    return embedded_rf_support, embedded_rf_feature

In [98]:
embedded_rf_support, embedded_rf_feature = embedded_rf_selector(X, y, num_feats)
print(str(len(embedded_rf_feature)), 'selected features')

23 selected features


In [99]:
embedded_rf_feature

['Crossing',
 'Finishing',
 'ShortPassing',
 'Dribbling',
 'LongPassing',
 'BallControl',
 'Acceleration',
 'SprintSpeed',
 'Agility',
 'Stamina',
 'Volleys',
 'FKAccuracy',
 'Reactions',
 'Balance',
 'ShotPower',
 'Strength',
 'LongShots',
 'Aggression',
 'Interceptions',
 'Weak Foot',
 'Preferred Foot_Left',
 'Body Type_Courtois',
 'Nationality_Slovenia']

## Tree based(Light GBM): SelectFromModel

In [92]:
from sklearn.feature_selection import SelectFromModel
from lightgbm import LGBMClassifier

In [93]:
def embedded_lgbm_selector(X, y, num_feats):

    lgbm=LGBMClassifier(n_estimators=500,learning_rate=0.05,num_leaves=32,colsample_bytree=0.2,reg_alpha=3,reg_lambda=1,min_split_gain=0.01,min_child_weight=40)
    embedded_lgbm_selector=SelectFromModel(lgbm,max_features=num_feats)
    
    X_norm=MinMaxScaler().fit_transform(X)

    embedded_lgbm_selector.fit(X_norm,y)
    embedded_lgbm_support=embedded_lgbm_selector.get_support()
    embedded_lgbm_feature=X.loc[:,embedded_lgbm_support].columns.tolist()       
    
    return embedded_lgbm_support, embedded_lgbm_feature

In [94]:
embedded_lgbm_support, embedded_lgbm_feature = embedded_lgbm_selector(X, y, num_feats)
print(str(len(embedded_lgbm_feature)), 'selected features')

30 selected features


In [95]:
embedded_lgbm_feature

['Crossing',
 'Finishing',
 'ShortPassing',
 'Dribbling',
 'LongPassing',
 'BallControl',
 'Acceleration',
 'SprintSpeed',
 'Agility',
 'Stamina',
 'Volleys',
 'FKAccuracy',
 'Reactions',
 'Balance',
 'ShotPower',
 'Strength',
 'LongShots',
 'Aggression',
 'Interceptions',
 'Weak Foot',
 'Preferred Foot_Left',
 'Preferred Foot_Right',
 'Position_CAM',
 'Position_CB',
 'Position_CDM',
 'Position_CF',
 'Position_CM',
 'Position_GK',
 'Position_LAM',
 'Position_LB']

## Putting all of it together: AutoFeatureSelector Tool

In [100]:
pd.set_option('display.max_rows', None)

# put all selection together
feature_selection_df = pd.DataFrame({'Feature':feature_name, 'Pearson':cor_support, 'Chi-2':chi_support, 'RFE':rfe_support, 'Logistics':embedded_lr_support,
                                    'Random Forest':embedded_rf_support, 'LightGBM':embedded_lgbm_support})

# count the selected times for each feature
feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)

# display the top 100
feature_selection_df = feature_selection_df.sort_values(['Total','Feature'] , ascending=False)
feature_selection_df.index = range(1, len(feature_selection_df)+1)
feature_selection_df.head(num_feats)

d:\anaconda\envs\ML-Fangyi\lib\site-packages\numpy\core\fromnumeric.py:84: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  return reduction(axis=axis, out=out, **passkwargs)


,Feature,Pearson,Chi-2,RFE,Logistics,Random Forest,LightGBM,Total
1,Volleys,True,True,True,True,True,True,6
2,ShortPassing,True,True,True,True,True,True,6
3,Reactions,True,True,True,True,True,True,6
4,LongPassing,True,True,True,True,True,True,6
5,Finishing,True,True,True,True,True,True,6
6,BallControl,True,True,True,True,True,True,6
7,Weak Foot,True,False,True,True,True,True,5
8,Strength,True,False,True,True,True,True,5
9,SprintSpeed,True,False,True,True,True,True,5
10,Nationality_Slovenia,True,True,True,True,True,False,5


## Can you build a Python script that takes dataset and a list of different feature selection methods that you want to try and output the best (maximum votes) features from all methods?

In [101]:
def preprocess_dataset(dataset_path):
    
    player_df = pd.read_csv("fifa19.csv")
    
    numcols = ['Overall', 'Crossing','Finishing',  'ShortPassing',  'Dribbling','LongPassing', 'BallControl', 'Acceleration','SprintSpeed', 'Agility',  'Stamina','Volleys','FKAccuracy','Reactions','Balance','ShotPower','Strength','LongShots','Aggression','Interceptions']
    catcols = ['Preferred Foot','Position','Body Type','Nationality','Weak Foot']
    
    player_df = player_df[numcols+catcols]
    
    traindf = pd.concat([player_df[numcols], pd.get_dummies(player_df[catcols])],axis=1)
    features = traindf.columns
    
    traindf = traindf.dropna()
    
    y = traindf['Overall']>=87
    X = traindf.drop('Overall',axis=1)
    
    num_feats=30    
    
    return X, y, num_feats

In [102]:
def autoFeatureSelector(dataset_path, methods=[]):
    # Parameters
    # data - dataset to be analyzed (csv file)
    # methods - various feature selection methods we outlined before, use them all here (list)
    def cor_selector(X, y,num_feats):
        cor_list = []
        feature_name = X.columns.tolist()
        for i in X.columns.tolist():
            cor = np.corrcoef(X[i], y)[0, 1]
            cor_list.append(cor)
        cor_list = [0 if np.isnan(i) else i for i in cor_list]
        cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))[-num_feats:]].columns.tolist()
        cor_support = [True if i in cor_feature else False for i in feature_name]
        return cor_support, cor_feature
    
    def chi_squared_selector(X, y, num_feats):
        X_norm=MinMaxScaler().fit_transform(X)
        chi_selector=SelectKBest(score_func=chi2, k=num_feats)
        chi_selector.fit(X_norm,y)
        chi_support=chi_selector.get_support()
        chi_feature=X.loc[:,chi_support].columns.tolist()    
        return chi_support, chi_feature
    
    
    def rfe_selector(X, y, num_feats):
        lr = LogisticRegression()
        rfe_selector=RFE(estimator=lr,n_features_to_select=num_feats,step=10,verbose=5)
        X_norm=MinMaxScaler().fit_transform(X)
        rfe_selector.fit(X_norm,y)
        rfe_support=rfe_selector.get_support()
        rfe_feature=X.loc[:,rfe_support].columns.tolist()    
        return rfe_support, rfe_feature
    
    
    def embedded_log_reg_selector(X, y, num_feats):
        embedded_lr_selector=SelectFromModel(LogisticRegression(penalty='none'),max_features=num_feats)
        X_norm=MinMaxScaler().fit_transform(X)
        embedded_lr_selector.fit(X_norm,y)
        embedded_lr_support=embedded_lr_selector.get_support()
        embedded_lr_feature=X.loc[:,embedded_lr_support].columns.tolist()    
        return embedded_lr_support, embedded_lr_feature

    
    def embedded_rf_selector(X, y, num_feats):
        embedded_rf_selector=SelectFromModel(RandomForestClassifier(n_estimators=100),max_features=num_feats)
        X_norm=MinMaxScaler().fit_transform(X)
        embedded_rf_selector.fit(X_norm,y)
        embedded_rf_support=embedded_rf_selector.get_support()
        embedded_rf_feature=X.loc[:,embedded_rf_support].columns.tolist()
        return embedded_rf_support, embedded_rf_feature
    
    def embedded_lgbm_selector(X, y, num_feats):
        lgbc=LGBMClassifier(n_estimators=500,learning_rate=0.05,num_leaves=32,colsample_bytree=0.2,reg_alpha=3,reg_lambda=1,min_split_gain=0.01,min_child_weight=40)
        embedded_lgbm_selector=SelectFromModel(lgbc,max_features=num_feats)
        X_norm=MinMaxScaler().fit_transform(X)
        embedded_lgbm_selector.fit(X_norm,y)
        embedded_lgbm_support=embedded_lgbm_selector.get_support()
        embedded_lgbm_feature=X.loc[:,embedded_lgbm_support].columns.tolist()    
        return embedded_lgbm_support, embedded_lgbm_feature
    
    
    
    # preprocessing
    X, y, num_feats = preprocess_dataset(dataset_path)
    
    # Run every method we outlined above from the methods list and collect returned best features from every method
    if 'pearson' in methods:
        cor_support, cor_feature = cor_selector(X, y,num_feats)
    if 'chi-square' in methods:
        chi_support, chi_feature = chi_squared_selector(X, y,num_feats)
    if 'rfe' in methods:
        rfe_support, rfe_feature = rfe_selector(X, y,num_feats)
    if 'log-reg' in methods:
        embedded_lr_support, embedded_lr_feature = embedded_log_reg_selector(X, y, num_feats)
    if 'rf' in methods:
        embedded_rf_support, embedded_rf_feature = embedded_rf_selector(X, y, num_feats)
    if 'lgbm' in methods:
        embedded_lgbm_support, embedded_lgbm_feature = embedded_lgbm_selector(X, y, num_feats)
    methods=['pearson','chi_square','rfe','log_reg','rf','lgbm']    
    
    # Combine all the above feature list and count the maximum set of features that got selected by all methods
    
    pd.set_option('display.max_rows', None)
    feature_selection_df = pd.DataFrame({'Feature':feature_name, 'Pearson':cor_support, 'Chi-2':chi_support, 'RFE':rfe_support, 'Logistics':embedded_lr_support,'Random Forest':embedded_rf_support, 'LightGBM':embedded_lgbm_support})
    feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)
    feature_selection_df = feature_selection_df.sort_values(['Total','Feature'] , ascending=False)
    feature_selection_df.index = range(1, len(feature_selection_df)+1)
    best_features=feature_selection_df['Feature'].head().tolist()    
    
    return best_features

In [103]:
best_features = autoFeatureSelector(dataset_path="fifa19.csv", methods=['pearson', 'chi-square', 'rfe', 'log-reg', 'rf', 'lgbm'])
best_features

Fitting estimator with 223 features.
Fitting estimator with 213 features.
Fitting estimator with 203 features.
Fitting estimator with 193 features.
Fitting estimator with 183 features.
Fitting estimator with 173 features.
Fitting estimator with 163 features.
Fitting estimator with 153 features.
Fitting estimator with 143 features.
Fitting estimator with 133 features.
Fitting estimator with 123 features.
Fitting estimator with 113 features.
Fitting estimator with 103 features.
Fitting estimator with 93 features.
Fitting estimator with 83 features.
Fitting estimator with 73 features.
Fitting estimator with 63 features.
Fitting estimator with 53 features.
Fitting estimator with 43 features.
Fitting estimator with 33 features.


d:\anaconda\envs\ML-Fangyi\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
d:\anaconda\envs\ML-Fangyi\lib\site-packages\numpy\core\fromnumeric.py:84: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  return reduction(axis=axis, out=out, **passkwargs)


['Reactions', 'LongPassing', 'Volleys', 'Strength', 'SprintSpeed']